In [ ]:
%reload_ext autoreload
%autoreload 2

from copy import copy
import os
from pathlib import Path
from pathlib import Path
import pickle

%matplotlib inline
import matplotlib.pyplot as plt
from bunch import Bunch
import numpy as np
import tensorflow as tf
import glob2 as glob
from azureml.core import Experiment, Workspace
from azureml.core.run import Run

from visualisation import render_plot
from exporter import export_obj
from constants import MASK_CHILD
from depthmap import Depthmap

REPO_DIR = Path('/Users/markus/Development/cgm/cgm-ml')
sys.path.append(str(REPO_DIR /'cgmml'))

from cgmml.common.model_utils.utils import download_dataset, get_dataset_path, AzureLogCallback, create_tensorboard_callback, get_optimizer, setup_wandb
from cgmml.common.model_utils.preprocessing import filter_blacklisted_qrcodes, preprocess_depthmap, preprocess_targets

In [ ]:
TOOLKIT_DIR = Path(os.getcwd()).absolute()
print(TOOLKIT_DIR)
calibration_fpath = TOOLKIT_DIR / 'tests' / 'huawei_p40pro' / 'camera_calibration.txt'

In [ ]:
DATASET_MODE_DOWNLOAD = "dataset_mode_download"
DATASET_MODE_MOUNT = "dataset_mode_mount"

CONFIG = Bunch(dict(
    DATASET_MODE=DATASET_MODE_DOWNLOAD,
    DATASET_NAME="anon-rgbd-5kscans",
    DATASET_NAME_LOCAL="anon-rgbd-5kscans-mini",  # 20 qrcodes
    SPLIT_SEED=0,
    IMAGE_TARGET_HEIGHT=240,
    IMAGE_TARGET_WIDTH=180,
    CODES=['100', '101', '102', '200', '201', '202'],
    NORMALIZATION_VALUE=7.5,
    TARGET_INDEXES=[0],  # 0 is height, 1 is weight.
))


def tf_load_pickle(path, max_value):
    def py_load_pickle(path, max_value):
        rgbd, targets = pickle.load(open(path.numpy(), "rb"))
        rgb = rgbd[0]  # shape: (240, 180, 3)
        depthmap = rgbd[1]  # shape: (240, 180)

        rgb = preprocess_depthmap(rgb)
        rgb = rgb / 255.

        depthmap = preprocess_depthmap(depthmap)
        depthmap = depthmap / max_value
        depthmap = tf.expand_dims(depthmap, -1)  # shape: (240, 180, 1)
        rgbd = tf.concat([rgb, depthmap], axis=2)
        rgbd = tf.image.resize(rgbd, (CONFIG.IMAGE_TARGET_HEIGHT, CONFIG.IMAGE_TARGET_WIDTH))
        targets = preprocess_targets(targets, CONFIG.TARGET_INDEXES)
        return rgbd, targets

    rgbd, targets = tf.py_function(py_load_pickle, [path, max_value], [tf.float32, tf.float32])
    rgbd.set_shape((CONFIG.IMAGE_TARGET_HEIGHT, CONFIG.IMAGE_TARGET_WIDTH, 4))
    targets.set_shape((len(CONFIG.TARGET_INDEXES,)))
    return rgbd, targets

def get_depthmap_files(paths):
    pickle_paths = []
    for path in paths:
        for code in CONFIG.CODES:
            pickle_paths.extend(glob.glob(os.path.join(path, code, "*.p")))
    return pickle_paths

In [ ]:
run = Run.get_context()
DATA_DIR = REPO_DIR / 'data' if run.id.startswith("OfflineRun") else Path(".")

# Offline run. Download the sample dataset and run locally. Still push results to Azure.
if run.id.startswith("OfflineRun"):
    workspace = Workspace.from_config()
    experiment = Experiment(workspace, "training-junkyard")
    run = experiment.start_logging(outputs=None, snapshot_directory=None)

    dataset_name = CONFIG.DATASET_NAME_LOCAL
    dataset_path = get_dataset_path(DATA_DIR, dataset_name)
    download_dataset(workspace, dataset_name, dataset_path)
else:
    assert False

dataset_path = os.path.join(dataset_path, "qrcode")
print(f'Dataset path: {dataset_path}')
print('Getting QR-code paths...')
qrcode_paths = glob.glob(os.path.join(dataset_path, "*"))
print(f'qrcode_paths: {len(qrcode_paths)}')
assert len(qrcode_paths) != 0

qrcode_paths = glob.glob(os.path.join(dataset_path, "*"))
print(f'qrcode_paths: {len(qrcode_paths)}')
assert len(qrcode_paths) != 0

paths_training = get_depthmap_files(qrcode_paths)

In [ ]:
path = paths_training[0]
path

In [ ]:
rgbd, target = tf_load_pickle(path, CONFIG.NORMALIZATION_VALUE)
rgbd_arr = rgbd.numpy()
bgr = rgbd_arr[:, :, :3]
rgb = bgr[:, :, ::-1]

depthmap = rgbd_arr[:, :, -1]
depthmap_rescaled = depthmap / depthmap.max() * 255

In [ ]:
dmap = Depthmap.create_from_array(depthmap_arr=depthmap, rgb_arr=rgb, calibration_fpath=str(calibration_fpath))

In [ ]:
floor = dmap.get_floor_level()
mask = dmap.segment_child(floor)

In [ ]:
mask.min(), mask.max()

In [ ]:
plt.imshow(mask, cmap='gray');

In [ ]:
mask_child = (mask==MASK_CHILD).astype(np.int)

In [ ]:
plt.imshow(mask_child, cmap='gray');

In [ ]:
plt.imshow(rgb);

In [ ]:
plt.imshow(depthmap, cmap='gray');

In [ ]:
plt.imshow(depthmap_rescaled, cmap='gray');

In [ ]:
print(rgb.shape, depthmap.shape)
rgd = copy(rgb)
rgd[:,:,-1] = depthmap_rescaled
plt.imshow(rgd);